In [ ]:
import os
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# np.set_printoptions(precision=2,suppress=True)

from osgeo import gdal,ogr,gdal_array
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import make_scorer
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve
import xarray as xr

In [ ]:
def get_raster(filename):
    data = gdal_array.LoadFile(filename).reshape(-1,1)
    df = pd.DataFrame(data)
    del data
    return df

In [ ]:
os.getcwd()
os.chdir("C:\\Users\\2019\\Desktop\\1a\\data")
os.getcwd()

In [ ]:
aspect = get_raster("factors/Aspect.tif")
elevation = get_raster("factors/elevation.tif")
faults = get_raster("factors/faults.tif")
landuse = get_raster("factors/landuse.tif")
lithology = get_raster("factors/lithology.tif")
loess = get_raster("factors/Loess.tif")
NDVI = get_raster("factors/NDVI.tif")
plan_curvature = get_raster("factors/plan_curvature.tif")
profile_curvature = get_raster("factors/profile_curvature.tif")
rainfall = get_raster("factors/rainfall.tif")
roads = get_raster("factors/raods.tif")
rivers = get_raster("factors/rivers.tif")
slope = get_raster("factors/slope_angle.tif")
twi = get_raster("factors/TWI.tif")
spi = get_raster("factors/SPI.tif")
landslides = get_raster("factors/landslide.tif")

In [ ]:
aspect.rename(columns={0:'aspect'},inplace=True)
elevation.rename(columns={0:'elevation'},inplace=True)
faults.rename(columns={0:'faults'},inplace=True)
landuse.rename(columns={0:'landuse'},inplace=True)
lithology.rename(columns={0:'lithology'},inplace=True)
loess.rename(columns={0:'loess'},inplace=True)
NDVI.rename(columns={0:'NDVI'},inplace=True)
plan_curvature.rename(columns={0:'plan_curvature'},inplace=True)
profile_curvature.rename(columns={0:'profile_curvature'},inplace=True)
rainfall.rename(columns={0:'rainfall'},inplace=True)
roads.rename(columns={0:'roads'},inplace=True)
rivers.rename(columns={0:'rivers'},inplace=True)
slope.rename(columns={0:'slope'},inplace=True)
twi.rename(columns={0:'twi'},inplace=True)
spi.rename(columns={0:'spi'},inplace=True)
landslides.rename(columns={0:'landslides'},inplace=True)

In [ ]:
aspect[aspect[:]<=-1]=0;
aspect[(aspect[:]>0) & (aspect[:]<=22.5) | (aspect[:]>337.5)] = 1.188
aspect[(aspect[:]>22.5) & (aspect[:]<=67.5)] = 1.083
aspect[(aspect[:]>67.5) & (aspect[:]<=112.5)] = 1.203
aspect[(aspect[:]>112.5) & (aspect[:]<=157.5)] = 0.857
aspect[(aspect[:]>157.5) & (aspect[:]<=202.5)] = 0.822
aspect[(aspect[:]>202.5) & (aspect[:]<=247.5)] = 0.693
aspect[(aspect[:]>247.5) & (aspect[:]<=292.5)] = 0.981
aspect[(aspect[:]>292.5) & (aspect[:]<=337.5)] = 1.164

In [ ]:
elevation[elevation[:]<=1000]=0.02;
elevation[(elevation[:]>1000) & (elevation[:]<=1200)] = 0.621
elevation[(elevation[:]>1200) & (elevation[:]<=1400)] = 2.489
elevation[(elevation[:]>1400) & (elevation[:]<=1600)] = 4.795
elevation[(elevation[:]>1600) & (elevation[:]<=1800)] = 3.588
elevation[(elevation[:]>1800) & (elevation[:]<=2000)] = 0.926
elevation[(elevation[:]>2000) & (elevation[:]<=2200)] = 0.387
elevation[(elevation[:]>2200)] = 0.106


In [ ]:
faults[faults[:]<=1000]=1.384;
faults[(faults[:]>1000) & (faults[:]<=2000)] = 1.423
faults[(faults[:]>2000) & (faults[:]<=3000)] = 1.264
faults[(faults[:]>3000) & (faults[:]<=4000)] = 0.937
faults[(faults[:]>4000) & (faults[:]<=5000)] = 0.389
faults[(faults[:]>5000)] = 0.230

In [ ]:
landuse[landuse[:]==1]=0
landuse[landuse[:]==2]=1.259
landuse[landuse[:]==3]=0
landuse[landuse[:]==4]=0.851
landuse[landuse[:]==5]=0.584

In [ ]:
lithology[lithology[:]==1]=1.261
lithology[lithology[:]==2]=0.576
lithology[lithology[:]==3]=0.852
lithology[lithology[:]==4]=0.735
lithology[lithology[:]==5]=5.626

In [ ]:
loess[loess[:]==1]=2.572
loess[loess[:]==2]=0.453

In [ ]:
NDVI[(NDVI[:]>0.08) & (NDVI[:]<=0.16)] = 0.47
NDVI[NDVI[:]<=0.08]= 0.143
NDVI[(NDVI[:]>0.39)] = 1.05
NDVI[(NDVI[:]>0.16) & (NDVI[:]<=0.24)] = 1.106
NDVI[(NDVI[:]>0.24) & (NDVI[:]<=0.31)] = 1.153
NDVI[(NDVI[:]>0.31) & (NDVI[:]<=0.39)] = 1.189

In [ ]:
plan_curvature[(plan_curvature[:]>1.07)] = 1.590
plan_curvature[plan_curvature[:]<=1.36]=1.459
plan_curvature[(plan_curvature[:]>0.23) & (plan_curvature[:]<=1.07)] = 1.506
plan_curvature[(plan_curvature[:]>-0.44) & (plan_curvature[:]<=0.23)] = 0.799
plan_curvature[(plan_curvature[:]>-1.36) & (plan_curvature[:]<=-0.44)] = 1.364

In [ ]:
profile_curvature[profile_curvature[:]<=-1.17] = 1.262
profile_curvature[(profile_curvature[:]>1.11)] = 1.711
profile_curvature[(profile_curvature[:]>-1.17) & (profile_curvature[:]<=-0.39)] = 1.651
profile_curvature[(profile_curvature[:]>-0.39) & (profile_curvature[:]<=0.24)] = 0.757
profile_curvature[(profile_curvature[:]>0.24) & (profile_curvature[:]<=1.11)] = 1.124

In [ ]:
rainfall[rainfall[:]<=300] = 0
rainfall[(rainfall[:]>300) & (rainfall[:]<=400)] = 0.045
rainfall[(rainfall[:]>400) & (rainfall[:]<=500)] = 0.43
rainfall[(rainfall[:]>500) & (rainfall[:]<=600)] = 1.51
rainfall[(rainfall[:]>600)] = 0.897

In [ ]:
roads[roads[:]<=400] = 0.821
roads[(roads[:]>400) & (roads[:]<=600)] = 1.111
roads[(roads[:]>600) & (roads[:]<=800)] = 1.454
roads[(roads[:]>800) & (roads[:]<=1000)] = 0.844
roads[(roads[:]>1000)] = 1.009

In [ ]:
rivers[rivers[:]<=200] = 1.119
rivers[(rivers[:]>200) & (rivers[:]<=400)] = 0.785
rivers[(rivers[:]>400) & (rivers[:]<=600)] = 1.06
rivers[(rivers[:]>600) & (rivers[:]<=800)] = 0.83
rivers[(rivers[:]>800) & (rivers[:]<=1000)] = 1.084
rivers[(rivers[:]>1000)] = 1.015

In [ ]:
slope[slope[:]<=10] = 0.285
slope[(slope[:]>10) & (slope[:]<=20)] = 1.372
slope[(slope[:]>20) & (slope[:]<=30)] = 1.766
slope[(slope[:]>30) & (slope[:]<=40)] = 1.400
slope[(slope[:]>40) & (slope[:]<=50)] = 1.414
slope[(slope[:]>50)] = 0

In [ ]:
twi[twi[:]<=5.31] = 1.569
twi[(twi[:]>5.31) & (twi[:]<=6.96)] = 1.084
twi[(twi[:]>6.95) & (twi[:]<=9.17)] = 0.408
twi[(twi[:]>9.17) & (twi[:]<=12.67)] = 0.319
twi[(twi[:]>12.67)] = 0.692

In [ ]:
spi[spi[:]<=20] = 0.734
spi[(spi[:]>20) & (spi[:]<=40)] = 1.363
spi[(spi[:]>40) & (spi[:]<=60)] = 1.103
spi[(spi[:]>60) & (spi[:]<=80)] = 1.952
spi[(spi[:]>80)] = 1.053

In [ ]:
#检查空值
null_value_stats = data.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]
# data.fillna(-999, inplace=True)#空值填充

In [ ]:
pred_data_LR = pd.concat([aspect,elevation,faults,landuse,lithology,loess,NDVI,plan_curvature,\
    profile_curvature,rainfall,roads,rivers,slope,twi,spi,landslides],axis=1)

pred_data_LR.head(2)